- `review_date` and `claim_date` is not always yyyy-mm-dd
    - Contains yyyy-mm-ddThh:mm:ss and None

In [1]:
from helper import download_prequential_dataset, download_dataset, download_article, explode_dictionary, clean_pd_df

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

DATASET_FP = "./WatClaimCheck_dataset" # CHANGE TO MATCH LOCAL

In [2]:
prequential_pd_df = download_prequential_dataset(DATASET_FP)

Extracting fields from metadata
Extracting fields from label
Start date is 2010-01-01. End date is 2021-07-01
Created 24 partitions


In [3]:
for i in prequential_pd_df:
    print(len(i))

382
656
704
699
807
795
618
592
701
740
675
841
1453
1869
1399
1367
1730
1973
2424
2617
3483
4088
2199
0
